In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [104]:
def gather_data(filename, sheet):
    df = pd.read_excel(filename,sheetname=sheet)
    #drop columns where all the values are na
    df = df.dropna(axis = 1, how = 'all')
    #naming the columns properly
    return df

def process_headers(df):
    df = df.copy()
    curr_colnames = list(df.columns)
    colnames =list(df.loc[0,:])
    for i,col in enumerate(list(curr_colnames)):
        if col.find("Unnamed") != -1:
            curr_colnames[i] = curr_colnames[i-1]
            col = curr_colnames[i-1]
        colnames[i]= str(col) + '__'+str(colnames[i]) 
    df.columns = colnames
    df = df.loc[1:,:]
    return df
        
def process_timeline(df):
    #aligns the dataframe with single timeline
    #break the dataframe into separate dataframes with regards to timelines
    df_dict = {}
    count_dates = [0 if x.lower().find('date')==-1 else 1 for x in list(df.columns)]
    ct_df = 0
    start = 0
    start_index = 0
    for i,dt in enumerate(count_dates):
        if dt == 1 and start ==0:
            df_dict[ct_df] = pd.DataFrame()
            start_index = i
            start = 1
            print start_index
        elif (dt ==1 and start == 1):
            df_dict[ct_df] = df.iloc[:,start_index:i]
            ct_df = ct_df + 1
            start_index = i
            start = 0
            print start_index, start
        elif i == len(count_dates) - 1 and start == 1:
            df_dict[ct_df] = df.iloc[:,start_index:]
            start = 0
    print df_dict.keys(), sum(count_dates)
    for key in sorted(list(df_dict.keys())):
        temp_df = df_dict[key].copy()
        temp_df = temp_df.loc[temp_df.iloc[:,0].dropna().index,:]
        temp_df.index = temp_df.iloc[:,0]
        temp_df = temp_df.iloc[:,1:]
        df_dict[key] = temp_df
        
    #merging the dataframes together
    total_df = np.nan
    for i,key in enumerate(sorted(list(df_dict.keys()))):
        if i == 0:
            total_df =df_dict[key]
        else:
            #print key, df_dict[key].head()
            total_df = total_df.merge(df_dict[key],left_index = True, right_index = True, how = 'outer')
    return total_df
    pass


In [83]:
def fly_calc(df, securities):
    #calculates the flies on 1-2-1 basis and gets the time-series of the fly
    pass
    return -1*np.array(df[securities[0]]) + 2*np.array(df[securities[1]]) + -1*np.array(df[securities[2]])

def fly_pca_calc(df, securities):
    temp_df = df[securities].dropna()
    X1 = np.array(temp_df[securities[0]])
    X2 = np.array(temp_df[securities[1]])
    X3 = np.array(temp_df[securities[2]])
    X = np.array(temp_df[securities])
    pca_model = PCA()
    pca_model.fit(X)
    weights = pca_model.components_[:,2]
    #normalize the weights
    weights = weights/weights[1]*2.
    #calc the fly
    return weights[0]*df[securities[0]]+weights[1]*df[securities[1]]+weights[2]*df[securities[2]]
    pass

#mean reversion equation - dxt = a(b-xt-1) + error
import sklearn as sk
from sklearn import linear_model
from sklearn.decomposition import PCA

def mean_reversion(flies):
    model_gaussian = linear_model.LinearRegression()
    dt_flies = flies - flies.shift(1)
    X = np.array(flies.shift(1).dropna()).reshape((-1,1))
    Y = np.array(dt_flies.dropna()).reshape((-1,1))
    model_gaussian.fit(X = X, y=Y)
    alpha = model_gaussian.coef_[0]*-1.
    theta = model_gaussian.intercept_/alpha
    var = model_gaussian.residues_
    return list(alpha)[0], list(theta)[0], list(var)[0]
    pass
    

In [3]:
filename = r'/Users/Abhishek/Desktop/Projects/Bond-Swap-Fly-monitor/Datafin.xlsx'
test_frame = pd.read_excel(filename, sheet = 'Sheet6')

In [4]:
test_frame = gather_data(filename,'Sheet6')
ret_frame = process_headers(test_frame)

In [105]:
ret_df = process_timeline(ret_frame)

[0, 1, 2, 3] 8


In [84]:
mean_reversion(ret_df.iloc[:,0].dropna())

/Users/Abhishek/anaconda/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function residues_ is deprecated; ``residues_`` is deprecated and will be removed in 0.19
  warnings.warn(msg, category=DeprecationWarning)


(0.003603625287131468, -0.76265435706818696, 0.36252122422544752)

In [100]:
ret_df.columns, ret_frame.columns

(Index([u'OBL 1 02/22/19 168 Govt__YLD_YTM_MID',
        u'OBL 1 02/22/19 168 Govt__ASSET_SWAP_SPD_MID',
        u'OBL 1 02/22/19 168 Govt__Z_SPRD_MID',
        u'DBR 2 01/04/2022 Govt__YLD_YTM_MID',
        u'DBR 2 01/04/2022 Govt__ASSET_SWAP_SPD_MID',
        u'DBR 2 01/04/2022 Govt__Z_SPRD_MID',
        u'DBR 0.5 02/15/26 Govt__YLD_YTM_MID',
        u'DBR 0.5 02/15/26 Govt__ASSET_SWAP_SPD_MID',
        u'DBR 0.5 02/15/26 Govt__Z_SPRD_MID',
        u'DBR 4 01/04/2037 Govt__YLD_YTM_MID',
        u'DBR 4 01/04/2037 Govt__ASSET_SWAP_SPD_MID',
        u'DBR 4 01/04/2037 Govt__Z_SPRD_MID'],
       dtype='object'), Index([u'OBL 1 02/22/19 168 Govt__Date',
        u'OBL 1 02/22/19 168 Govt__YLD_YTM_MID',
        u'OBL 1 02/22/19 168 Govt__ASSET_SWAP_SPD_MID',
        u'OBL 1 02/22/19 168 Govt__Z_SPRD_MID',
        u'DBR 3.25 01/04/20 09 Govt__Date',
        u'DBR 3.25 01/04/20 09 Govt__YLD_YTM_MID',
        u'DBR 3.25 01/04/20 09 Govt__ASSET_SWAP_SPD_MID',
        u'DBR 3.25 01/04/20 09 Gov